In [59]:
import random
import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import *
from tensorflow.keras.models import Model
from tensorflow.keras import losses
from keras import optimizers

matplotlib.use('TkAgg')

In [207]:
data = pd.read_csv("data/cation/DataBase_RF_bi_new_2.csv")

In [81]:
data

,name,Al,Si,Cation #1,Cation #2,Radius #1,Radius #2,Polarizability #1,Polarizability #2,Charge #1,Charge #2,S1 #1,S1' #1,S2 #1,S3 #1,S1 #2,S1' #2,S2 #2,S3 #2
0,Structure: Al_15_Si_177_Boulfelfel_Cs_4_K_11,15,177,4,11,1.67,1.38,2.420,0.841,1,1,0,2,2,0,1,0,8,2
1,Structure: Al_15_Si_177_Boulfelfel_Cs_6_K_9,15,177,6,9,1.67,1.38,2.420,0.841,1,1,0,3,2,1,1,0,6,2
2,Structure: Al_15_Si_177_Boulfelfel_K_1_Li_14,15,177,1,14,1.38,0.76,0.841,0.029,1,1,1,0,0,0,0,0,10,4
3,Structure: Al_15_Si_177_Boulfelfel_K_2_Rb_13,15,177,2,13,1.38,1.52,0.841,1.417,1,1,1,0,1,0,0,2,9,2
4,Structure: Al_15_Si_177_Boulfelfel_K_8_Na_7,15,177,8,7,1.38,1.02,0.841,0.181,1,1,1,2,5,0,0,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1950,Structure: Al_96_Si_96_Boulfelfel_Na_78_Rb_18,96,96,78,18,1.02,1.52,0.181,1.417,1,1,0,32,30,16,0,0,2,16
1951,Structure: Al_96_Si_96_Boulfelfel_Rb_1_Na_95,96,96,1,95,1.52,1.02,1.417,0.181,1,1,0,0,0,1,0,32,32,31
1952,Structure: Al_96_Si_96_Boulfelfel_Rb_2_Na_94,96,96,2,94,1.52,1.02,1.417,0.181,1,1,0,0,0,2,0,32,32,30
1953,Structure: Al_96_Si_96_Boulfelfel_Rb_58_K_38,96,96,58,38,1.52,1.38,1.417,0.841,1,1,5,4,19,30,11,3,20,4


In [208]:
data = data.loc[(data["Cation #1"] == (data["S1 #1"] + data["S1\' #1"] + data["S2 #1"] + data["S3 #1"]))
                &
                (data["Cation #2"] == (data["S1 #2"] + data["S1\' #2"] + data["S2 #2"] + data[
                    "S3 #2"]))]  #["Cation #1", "S1 #1", "S1\' #1", "S2 #1", "S3 #1"]]

In [63]:
data_without_names = data.drop(columns=['name'])
plt.matshow(data_without_names.corr())
continuous_features = data_without_names.describe().columns
plt.xticks(range(len(continuous_features)), continuous_features)
plt.yticks(range(len(continuous_features)), continuous_features)
plt.colorbar()
plt.show()

In [64]:
features = data.drop(columns=["name", "S1 #1", "S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"],
                     axis=1)
labels = data[["S1 #1", "S1\' #1", "S2 #1", "S3 #1", "S1 #2", "S1\' #2", "S2 #2", "S3 #2"]]

In [212]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.001,
                                                                            random_state=0)

In [213]:
labels_test

,S1 #1,S1' #1,S2 #1,S3 #1,S1 #2,S1' #2,S2 #2,S3 #2
1011,2,6,12,4,0,6,15,11
980,1,0,0,0,3,9,31,11


In [9]:
from sklearn.preprocessing import MinMaxScaler
# 
# scale_features_mm = MinMaxScaler()
# features_train = scale_features_mm.fit_transform(features_train)
# features_test = scale_features_mm.transform(features_test)
# 
# scale_labels_mm = MinMaxScaler()
# labels_train = scale_labels_mm.fit_transform(labels_train)
# labels_test = scale_labels_mm.transform(labels_test)

In [218]:
def mps_loss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    delta = tf.abs(y_true - y_pred)
    delta_ = tf.reduce_mean(delta)
    sigma = tf.sqrt(tf.reduce_mean(tf.square(delta - delta_)))
    size = tf.size(delta, out_type=tf.int32)
    return tf.reduce_mean(delta_ * (sigma / tf.cast(size, tf.float32) + 1))






def set_model(adapt_data):
    #shared
    inputs = Input(shape=3, name="struct_input")
    x = Dense(3, activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(3, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(3, activation='relu')(x)
    feature_ext_model = Model(inputs, x)
    #
    struct_res = Input(shape=(6,))
    x = Dense(9, activation='relu')(struct_res)
    #x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Dense(4, activation='relu')(x)
    struct_res_model = Model(struct_res, x)
    #
    
    input1, input2 = Input(shape=(3,)), Input(shape=(3,))
    normalization_layer = Normalization()
    normalization_layer.adapt(adapt_data)
    input1 = normalization_layer(input1)
    input2 = normalization_layer(input2)
    num_elements1 = Input(shape=1)
    num_elements2 = Input(shape=1)
    out1, out2 = feature_ext_model(input1), feature_ext_model(input2)
    y = Average()([out1, out2])
    struct1_state = Concatenate()([y, input1])
    struct2_state = Concatenate()([y, input2])
    struct1_state = struct_res_model(struct1_state)
    struct2_state = struct_res_model(struct2_state)
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct1_state)
    output1 = Multiply()([l1, num_elements1])
    l2 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct2_state)
    output2 = Multiply()([l2, num_elements2])
    out = Concatenate()([output1, output2])
    model = Model([input1, input2, num_elements1, num_elements2], out)
    model.compile(loss='mse', optimizer='Adam')
    return model


import tensorflow as tf

def set_model2(adapt_data):
    #
    struct1, struct2 = Input(shape=(3,)), Input(shape=(3,))
    normalization_layer = Normalization()
    normalization_layer.adapt(adapt_data)
    struct1 = normalization_layer(struct1)
    struct2 = normalization_layer(struct2)
    inputs = Concatenate()([struct1, struct2])
    x = Dense(6, activation='relu', name="struct_d1")(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation='relu')(x)
    #x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(6, activation='relu')(x)
    struct_model = Model([struct1, struct2], x)
    #
    #
    struct_res = Input(shape=(9,))
    x = Dense(9, activation='relu', name="struct_d2")(struct_res)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(9, activation='relu')(x)
    #x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    x = Dense(4, activation='relu')(x)
    struct_res_model = Model(struct_res, x)
    #
    input1, input2 = Input(shape=(3,)), Input(shape=(3,))
    num_elements1 = Input(shape=1)
    num_elements2 = Input(shape=1)
    out1, out2 = struct_model([input1, input2]), struct_model([input2, input1])
    y = Average()([out1, out2])
    struct1_state = Concatenate()([y, input1])
    struct2_state = Concatenate()([y, input2])
    struct1_state = struct_res_model(struct1_state)
    struct2_state = struct_res_model(struct2_state)
    
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct1_state)
    output1 = Multiply()([l1, num_elements1])
    l2 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(struct2_state)
    output2 = Multiply()([l2, num_elements2])
    out = Concatenate()([output1, output2])
    
    model = Model([input1, input2, num_elements1, num_elements2], out)
    model.compile(loss=mps_loss, optimizer=optimizers.Adam(clipnorm=0.1)) # losses.Huber(delta=5.0)
    return model



def create_model(x):
    inputs = Input(shape=6)
    num_elements = Input(shape=1)
    x = Dense(10, activation='relu')(inputs)
    x = Dropout(0.3)(x)
    d = Dense(8, activation='relu')(x)
    l1 = Lambda(lambda x: x / tf.reduce_sum(x, axis=1, keepdims=True))(d)
    outputs = Multiply()([l1, num_elements])
    model = tf.keras.Model(inputs=[inputs, num_elements], outputs=outputs)
    model.compile(loss="mse", optimizer=optimizers.Adam())
    return model


model = set_model(np.concatenate((f_train[0], f_train[1])))
#model = set_model2(np.concatenate((f_train[0], f_train[1])))

In [215]:
def generate_data_for_set():
    num_train = [[], []]
    num_test = [[], []]
    f_train = [[], []]
    f_test = [[], []]
    for i in range(len(features_train)):
        f_train[0].append(features_train[["Cation #1", "Radius #1", "Polarizability #1"]].iloc[i])
        f_train[1].append(features_train[["Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
        num_train[0].append(features_train[["Cation #1"]].iloc[i])
        num_train[1].append(features_train[["Cation #2"]].iloc[i])
    for i in range(len(features_test)):
        f_test[0].append(features_test[["Cation #1", "Radius #1", "Polarizability #1"]].iloc[i])
        f_test[1].append(features_test[["Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
        num_test[0].append(features_test[["Cation #1"]].iloc[i])
        num_test[1].append(features_test[["Cation #2"]].iloc[i])

    f_train[0] = np.array(f_train[0])
    f_train[1] = np.array(f_train[1])
    f_test[0] = np.array(f_test[0])
    f_test[1] = np.array(f_test[1])
    
    num_train[0] = np.array(num_train[0])
    num_train[1] = np.array(num_train[1])
    num_test[0] = np.array(num_test[0])
    num_test[1] = np.array(num_test[1])
    return f_train, f_test, num_train, num_test

f_train, f_test, num_train, num_test = generate_data_for_set()

# f_train = []
# f_test = []
# num_train = []
# num_test = []
# for i in range(len(features_train)):
#     #labels_train.iloc[i] /= np.sum(labels_train.iloc[i])
#     f_train.append(features_train[["Cation #1", "Radius #1", "Polarizability #1", "Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
#     num_train.append(features_train.iloc[i]["Al"])
# for i in range(len(features_test)):
#     #labels_test.iloc[i] /= np.sum(labels_test.iloc[i])
#     f_test.append(features_test[["Cation #1", "Radius #1", "Polarizability #1", "Cation #2", "Radius #2", "Polarizability #2"]].iloc[i])
#     num_test.append(features_test.iloc[i]["Al"])

In [86]:
# model.compile(optimizer='adam', loss=mps_loss)
from keras import backend as K
K.set_value(model.optimizer.learning_rate, 0.01)
model.compile(loss=mps_loss, optimizer=optimizers.Adam())

In [219]:
def plot():
    plt.legend()
    plt.grid(True)
    plt.show()


def plot_loss(history):
    plt.plot(history.history['loss'], label='loss', marker=".")
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Error [MPG]')
    plot()


mcp_save = tf.keras.callbacks.ModelCheckpoint(filepath='data/models/mps3.keras', save_best_only=True,
                                              monitor='val_loss', mode='min', verbose=1, save_weights_only=False,
                                              save_freq='epoch')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                                      patience=1000, verbose=1, mode='auto')

# history = model.fit([np.array(f_train), np.array(num_train)], labels_train,
#                     epochs=100, batch_size=16, shuffle=True,
#                     validation_data=([np.array(f_test), np.array(num_test)], labels_test), callbacks=[mcp_save])

# history = model.fit((f_train[0], f_train[1], num_train[0], num_train[1]), labels_train,
#                     epochs=2000, batch_size=64, shuffle=True,
#                     validation_data=((f_test[0], f_test[1], num_test[0], num_test[1]), labels_test), callbacks=[mcp_save, reduce_lr_loss])

history = model.fit((f_train[0], f_train[1], num_train[0], num_train[1]), labels_train,
                    epochs=500, batch_size=64, shuffle=True,
                    validation_data=((f_ref[0], f_ref[1], num_ref[0], num_ref[1]), labels_ref), callbacks=[mcp_save, reduce_lr_loss])

plot_loss(history)

Epoch 1/500
31/31 [==============================] - ETA: 0s - loss: nan
Epoch 1: val_loss improved from inf to 243.13960, saving model to data/models\mps3.keras
31/31 [==============================] - 11s 131ms/step - loss: nan - val_loss: 243.1396 - lr: 0.0010
Epoch 2/500
31/31 [==============================] - ETA: 0s - loss: nan
Epoch 2: val_loss improved from 243.13960 to 143.76718, saving model to data/models\mps3.keras
31/31 [==============================] - 4s 123ms/step - loss: nan - val_loss: 143.7672 - lr: 0.0010
Epoch 3/500
31/31 [==============================] - ETA: 0s - loss: nan
Epoch 3: val_loss improved from 143.76718 to 110.06128, saving model to data/models\mps3.keras
31/31 [==============================] - 4s 125ms/step - loss: nan - val_loss: 110.0613 - lr: 0.0010
Epoch 4/500
31/31 [==============================] - ETA: 0s - loss: nan
Epoch 4: val_loss improved from 110.06128 to 99.60011, saving model to data/models\mps3.keras
31/31 [========================

In [189]:
model = tf.keras.models.load_model('data/models/mps3.keras', custom_objects={'mps_loss': mps_loss})

In [190]:
def process_prediction(prediction, num1, num2):
    return np.concatenate((prediction[:4] * num1, prediction[4:] * num2))


#predictions = model.predict([np.array(f_test), np.array(num_test)])
predictions = model.predict((f_test[0], f_test[1], num_test[0], num_test[1]))
predictions_2 = model.predict((f_test[1], f_test[0], num_test[1], num_test[0]))

#predictions = [process_prediction(predictions[i], num_test[0][i], num_test[1][i]) for i in range(len(predictions))]
#labels_test = [process_prediction(labels_test[i], num_test[0][i], num_test[1][i]) for i in range(len(labels_test))]

10/10 [==============================] - 0s 25ms/step


In [191]:
print(features_test.iloc[i])
print(labels_test.iloc[i])

Al                    90.000
Si                   102.000
Cation #1             58.000
Cation #2             32.000
Radius #1              1.520
Radius #2              1.020
Polarizability #1      1.417
Polarizability #2      0.181
Charge #1              1.000
Charge #2              1.000
Name: 1814, dtype: float64
S1 #1      6
S1' #1     9
S2 #1     12
S3 #1     31
S1 #2      6
S1' #2     4
S2 #2     21
S3 #2      1
Name: 1814, dtype: int64


In [192]:
def mps(first:np.array, second:np.array):
    delta = np.abs(first - second)
    delta_ = np.mean(delta)
    sigma = np.sqrt(np.mean(np.abs(delta - delta_)))
    N = len(delta)
    return np.mean(delta_/N * (sigma/N + 1))



i = random.randint(0, len(predictions) - 1)
print("№", i)
rounded_prediction = [round(p, 1) for p in predictions[i]]
print(rounded_prediction)
rounded_prediction2 = [round(p, 1) for p in predictions_2[i]]
#print(rounded_prediction2)

rounded_real = [round(float(p), 1) for p in np.array(labels_test.iloc[i])]
print(rounded_real)
print(sum(predictions[i]), sum(labels_test.iloc[i]))

№ 126
[1.4, 5.4, 14.5, 4.6, 3.7, 3.3, 12.9, 10.0]
[0.0, 8.0, 12.0, 6.0, 3.0, 4.0, 15.0, 8.0]
56.00000071525574 56


In [193]:
mps_list = []
for i in range(len(predictions)):
    p = predictions[i]
    real = np.array(labels_test.iloc[i])
    mps_list.append(mps(p, real))
print(sum(mps_list))
print(sum(mps_list)/len(mps_list))

81.47309030728226
0.27997625535148546


In [74]:
plt.hist(mps_list)
plt.show()

In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import textwrap
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau
def corr_heatmap_with_pval(df, method = 'pearson', figsize=(20, 10), title=None):
  """
  df: dataframe to be used. Ensured the dataframe has been sliced to contain only the column you need. It accepts only numerical columns
  method: default uses the pearson method. It overall permits 3 methods; 'pearson', 'spearman' and 'kendall'
  figsize: default is (20, 10) but you can change it based on your preference
  title: Specify the title for your chart, default is None
  """
  # Make a copy of the df
  data = df.copy()
  # Check features correlation
  corr = data.corr(method = method)

  # Create a mask to hide the upper triangle
  mask = np.zeros_like(corr, dtype=bool)
  mask[np.triu_indices_from(mask)] = True

  # Set the diagonal elements of the mask to False to display self-correlation
  np.fill_diagonal(mask, False)

  fig, ax = plt.subplots(figsize=figsize)
  fig.tight_layout()
  plt.title(title, fontsize=14)

  # Create the heatmap with the custom mask
  heatmap = sns.heatmap(corr,
                        annot=True,
                        annot_kws={"fontsize": 10},  # Adjust annotation font size
                        fmt='.2f',
                        linewidths=0.5,
                        cmap='RdBu',
                        mask=mask,
                        ax=ax)

  # Create a function to calculate and format p-values
  p_values = np.full((corr.shape[0], corr.shape[1]), np.nan)
  for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[1]):
      x = data.iloc[:, i]
      y = data.iloc[:, j]
      mask = ~np.logical_or(np.isnan(x), np.isnan(y))
      if np.sum(mask) > 0:
        if method == 'pearson':
          p_values[i, j] = pearsonr(x[mask], y[mask])[1] #Changes based on the method chosen in the function
        elif method == 'kendall':
          p_values[i, j] = kendalltau(x[mask], y[mask])[1]
        elif method == 'spearman':
          p_values[i, j] = spearmanr(x[mask], y[mask])[1]
  
  p_values = pd.DataFrame(p_values, columns=corr.columns, index=corr.index)

  # Create a mask for the p-values heatmap
  mask_pvalues = np.triu(np.ones_like(p_values), k=1)

  # Calculate the highest and lowest correlation coefficients
  max_corr = np.max(corr.max())
  min_corr = np.min(corr.min())
  
  # Annotate the heatmap with p-values and change text color based on correlation value
  for i in range(p_values.shape[0]):
    for j in range(p_values.shape[1]):
      if mask_pvalues[i, j]:
        p_value = p_values.iloc[i, j]
        if not np.isnan(p_value):
          correlation_value = corr.iloc[i, j]
          text_color = 'white' if correlation_value >= (max_corr - 0.4) or correlation_value <= (min_corr + 0.4) else 'black'
          if p_value <= 0.01:
            #include double asterisks for p-value <= 0.01
            ax.text(i + 0.5, j + 0.8, f'(p = {p_value:.2f})**',
                    horizontalalignment='center',
                    verticalalignment='center',
                    fontsize=8,
                    color=text_color)
          elif p_value <= 0.05:
            #include single asterisks for p-value <= 0.05
            ax.text(i + 0.5, j + 0.8, f'(p = {p_value:.2f})*',
                    horizontalalignment='center',
                    verticalalignment='center',
                    fontsize=8,
                    color=text_color)
          else:
            ax.text(i + 0.5, j + 0.8, f'(p = {p_value:.2f})',
                    horizontalalignment='center',
                    verticalalignment='center',
                    fontsize=8,
                    color=text_color)

  # Customize x-axis labels
  x_labels = [textwrap.fill(label.get_text(), 12) for label in ax.get_xticklabels()]
  ax.set_xticklabels(x_labels, rotation=0, ha="center")

  # Customize y-axis labels
  y_labels = [textwrap.fill(label.get_text(), 15) for label in ax.get_yticklabels()]
  ax.set_yticklabels(y_labels, rotation=0, ha="right")
  ax.grid(False)
  plt.show()

In [12]:
# extended_data = pd.DataFrame(columns=["Al", "Si", "Cation #1", "Cation #2", "Radius #1", "Radius #2", "Polarizability #1", "Polarizability #2",
#                                       "Charge #1", "Charge #2", "S1 #1", "S1\' #1", "S2 #1", "S3 #1",
#                                       "S1 #2", "S1\' #2", "S2 #2", "S3 #2"])
for index, row in data.iterrows():
    data.loc[len(data)] = [row["name"], row["Al"], row["Si"],
                                             row["Cation #2"], row["Cation #1"],
                                             row["Radius #2"], row["Radius #1"],
                                             row["Polarizability #2"], row["Polarizability #1"],
                                             row["Charge #2"], row["Charge #1"],
                                             row["S1 #2"], row["S1\' #2"], row["S2 #2"], row["S3 #2"],
                                             row["S1 #1"], row["S1\' #1"], row["S2 #1"], row["S3 #1"]]
data = data.drop(columns=["name"])

In [16]:
corr_heatmap_with_pval(data, method ='kendall', figsize=(10, 5))

In [53]:
len(data)

1000

In [77]:
reference_data = pd.read_csv('data/cation/Reference_Data_Base_2024_fp.csv', delimiter=',', header=None,)
reference_data = reference_data.T
new_header = reference_data.iloc[0]
reference_data = reference_data[1:]
reference_data.columns = new_header
reference_data

,Cation #1,Number of Cations #1 per UC,Charge of Cation #1,Cation #2,Number of Cations #2 per UC,Charge of Cation #2,Cation #3,Number of cations #3 per UC,Charge of Cation #3,Number of Al atoms per UC,...,Charge balance (True/False),Data source (NMR/XRD),Dehydrated (yes/no),Dehydration temperature (in K),Evacuated (yes/no),Evacuation pressure (in Torr),Crystallography structure (yes/no),Average UC length (in Angstroms),Doi:,Comments
1,K,48.2,1,-,-,-,-,-,-,48.2,...,True,XRD,yes,573,yes,1.00E-5,yes,24.776,https://doi.org/10.1021/j100649a008,NaN
2,K,54.7,1,-,-,-,-,-,-,54.7,...,True,XRD,yes,573,yes,1.00E-5,yes,24.896,https://doi.org/10.1021/j100649a008,NaN
3,K,69.8,1,-,-,-,-,-,-,69.8,...,True,XRD,yes,573,yes,1.00E-5,yes,24.973,https://doi.org/10.1021/j100649a008,NaN
4,K,86.5,1,-,-,-,-,-,-,86.5,...,True,XRD,yes,573,yes,1.00E-5,yes,25.076,https://doi.org/10.1021/j100649a008,NaN
5,Na,93,1,-,-,-,-,-,-,93,...,True,NMR,yes,673,yes,8.00E-5,no,-,https://doi.org/10.1021/ja0102538,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,Ca,28,2,NaN,NaN,NaN,NaN,NaN,NaN,56.0,...,True,XRD,yes,748,no,NaN,yes,24.71,https://doi.org/10.1016/0025-5408(68)90112-8,NaN
107,Ca,25,2,NaN,NaN,NaN,NaN,NaN,NaN,50.0,...,True,XRD,yes,520,yes,7.50E-6,no,24.7,https://doi.org/10.1016/S0167-2991(08)64089-6,NaN
108,Sr,46,2,NaN,NaN,NaN,NaN,NaN,NaN,92.0,...,True,XRD,yes,633,yes,2.00E-6,no,25.214,https://doi.org/10.1016/S1387-1811(99)00012-8,NaN
109,Sr,32,2,Rb,16,1,Na,4,1,84.0,...,True,XRD,yes,573,no,NaN,no,25.15,https://doi.org/10.1016/S0167-2991(04)80689-X,11.2 molecules H2O per UC


In [155]:
bi_cation_data = reference_data.loc[(reference_data["Cation #1"].notnull()) & (reference_data["Cation #1"] != "-") &
                          (reference_data["Cation #2"].notnull()) & (reference_data["Cation #2"] != "-") &
                          (reference_data["Cation #3"].isnull() | (reference_data["Cation #3"] == "-"))]
bi_cation_data

,Cation #1,Number of Cations #1 per UC,Charge of Cation #1,Cation #2,Number of Cations #2 per UC,Charge of Cation #2,Cation #3,Number of cations #3 per UC,Charge of Cation #3,Number of Al atoms per UC,...,Charge balance (True/False),Data source (NMR/XRD),Dehydrated (yes/no),Dehydration temperature (in K),Evacuated (yes/no),Evacuation pressure (in Torr),Crystallography structure (yes/no),Average UC length (in Angstroms),Doi:,Comments
16,K,52,1,Na,39,1,NaN,NaN,NaN,91.0,...,True,XRD,yes,673,no,760,NaN,24.861,https://doi.org/10.1016/s0144-2449(05)80234-9,NaN
17,Cs,45,1,Na,47,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,673,yes,2.00E-6,yes,24.898,https://doi.org/10.1016/j.micromeso.2004.01.006,NaN
19,Co,38.1,2,Na,15.8,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,673,yes,1.00E-3,yes,24.597,https://doi.org/10.1016/j.micromeso.2007.12.031,NaN
20,Cs,45.9,1,Na,37.6,1,NaN,NaN,NaN,83.5,...,True,NMR,yes,673,yes,7.50E-5,no,NaN,https://doi.org/10.1021/jp001571g,NaN
21,Ca,19,2,Na,48,1,NaN,NaN,NaN,86.0,...,True,XRD,yes,773,yes,NaN,no,NaN,https://doi.org/10.1016/j.micromeso.2007.01.043,NaN
24,Co,34,2,Ca,9,2,NaN,NaN,NaN,86.0,...,True,XRD,yes,773,no,NaN,NaN,NaN,https://doi.org/10.1016/j.micromeso.2007.01.043,NaN
34,Cs,44,1,Na,47,1,NaN,NaN,NaN,91.0,...,True,XRD,yes,673,no,7.60E+2,yes,24.98,10.1016/s0144-2449(05)80234-9,NaN
36,Li,86,1,Na,6,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,623,yes,1.00E-6,NaN,NaN,https://doi.org/10.7745/KJSSF.2013.46.4.260,NaN
37,Li,87,1,Na,5,1,NaN,NaN,NaN,92.0,...,True,XRD,yes,623,yes,1.00E-6,NaN,NaN,https://doi.org/10.7745/KJSSF.2013.46.4.261,NaN
38,Li,81,1,Na,15,1,NaN,NaN,NaN,96.0,...,True,NMR,yes,296,NaN,7.50E-5,NaN,NaN,https://doi.org/10.1016/S1387-1811(98)00211-X,NaN


In [84]:
cation_db = pd.read_csv('data/cation/cations_params.csv')
cation_db

,Cation,Radius,Polarizability
0,Li,0.76,0.029
1,Na,1.02,0.181
2,K,1.38,0.841
3,Rb,1.52,1.417
4,Cs,1.67,2.420
5,Ag,1.15,1.360
6,Tl,1.50,0.940
7,Ca,1.00,0.472
8,Mg,0.72,0.094
9,Ba,1.35,1.563


In [174]:
def generate_data_for_reference(data_ref: pd.DataFrame, cation_db: pd.DataFrame):
    num_ref = [[], []]
    f_ref = [[], []]
    labels_ref = []
    cations_type = []
    
    for index, row in data_ref.iterrows():
        cation_number1 = float(row["Number of Cations #1 per UC"])
        cation_number2 = float(row["Number of Cations #2 per UC"])
        num_ref[0].append(cation_number1)
        num_ref[1].append(cation_number2)
        radius1 = cation_db[cation_db["Cation"] == row["Cation #1"]]["Radius"].values[0]
        radius2 = cation_db[cation_db["Cation"] == row["Cation #2"]]["Radius"].values[0]
        polarizability1 = cation_db[cation_db["Cation"] == row["Cation #1"]]["Polarizability"].values[0]
        polarizability2 = cation_db[cation_db["Cation"] == row["Cation #2"]]["Polarizability"].values[0]
        f_ref[0].append([cation_number1, radius1, polarizability1])
        f_ref[1].append([cation_number2, radius2, polarizability2])
        
        labels_ref.append([row["SI site occupation by Cation #1"], row["SI\' site occupation by Cation #1"],
                           row["SII site occupation by Cation #1"], row["SIII site occupation by Cation #1"],
                           
                           row["SI site occupation by Cation #2"], row["SI\' site occupation by Cation #2"],
                           row["SII site occupation by Cation #2"], row["SIII site occupation by Cation #2"]])
        cations_type.append([[row["Cation #1"]], [row["Cation #2"]]])
    f_ref[0] = np.array(f_ref[0])
    f_ref[1] = np.array(f_ref[1])
    num_ref[0] = np.array(num_ref[0])
    num_ref[1] = np.array(num_ref[1])
    labels_ref = np.array(labels_ref)
    return f_ref, num_ref, labels_ref, cations_type

f_ref, num_ref, labels_ref, cations_type = generate_data_for_reference(bi_cation_data, cation_db)
labels_ref = labels_ref.astype(float)

In [168]:
cation = bi_cation_data.iloc[1]["Cation #1"]
print(cation)
print(cation_db[cation_db["Cation"] == cation]["Radius"].values[0])
print(f_ref[0][1])

Cs
1.67
[45.    1.67  2.42]


In [220]:
predictions_ref = model.predict((f_ref[0], f_ref[1], num_ref[0], num_ref[1]))

2/2 [==============================] - 2s 129ms/step


In [225]:
i = random.randint(0, len(predictions_ref) - 1)
print("№", i)
rounded_prediction = [round(p, 1) for p in predictions_ref[i]]
print(rounded_prediction)

rounded_real = [round(float(p), 1) for p in np.array(labels_ref[i])]
print(rounded_real)
print(sum(predictions_ref[i]), sum(labels_ref[i]))
print("mps: ", mps(predictions_ref[i], labels_ref[i]))
print(f_ref[0][i])
print(cations_type[i])

№ 31
[2.4, 2.5, 13.0, 5.2, 13.5, 6.6, 30.6, 18.3]
[15.7, 0.0, 7.4, 0.0, 0.0, 17.7, 24.0, 27.2]
91.99999952316284 92.0
mps:  1.2775419515186626
[23.    1.15  1.36]
[['Ag'], ['Tl']]


In [222]:
mps_list = []
for i in range(len(predictions_ref)):
    p = predictions_ref[i]
    real = np.array(labels_ref[i])
    mps_list.append(mps(p, real))
print(sum(mps_list))
print(sum(mps_list)/len(mps_list))
plt.hist(mps_list)
plt.show()

42.82590133481233
0.8397235555845555
